In [8]:
import pandas as pd

file_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\데이터검증\테스트.xlsx"
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()  # 앞뒤 공백 제거

df2 = df[df["t(Entry)"] != -1]
display(df2)

excel_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\데이터검증\STVM 검증2.xlsx"
df2.to_excel(excel_path, index=False)

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],b[m/s2],Occ,Pers,tQueue,VehLength[m]
6,30023,1800.07,-1.0,761,100,0,107.5,0.00,0.03,1,0,4.21
7,30061,1800.01,-1.0,729,100,0,113.8,-0.25,0.09,1,0,4.76
8,10118,1800.13,-1.0,427,300,0,91.1,0.00,0.07,1,0,12.40
9,20012,1800.14,-1.0,873,100,0,104.0,0.00,0.06,1,0,4.01
11,20030,1800.19,-1.0,757,100,0,111.6,0.00,0.01,1,0,4.76
...,...,...,...,...,...,...,...,...,...,...,...,...
425774,30051,5399.89,-1.0,2523,100,0,94.9,-0.42,0.01,1,0,4.61
425778,10047,5399.94,-1.0,2528,100,0,116.7,0.98,0.06,1,0,4.21
425783,20099,5399.91,-1.0,2429,300,0,100.0,0.00,0.09,1,0,12.40
425786,30052,5399.99,-1.0,2501,100,0,101.2,-0.35,0.01,1,0,4.01
